In [2]:
import sys
import os
import pandas as pd 
import numpy as np
import os 
import sys 
from code.data_processing import get_all_datasets
import matplotlib.pyplot as plt

from ResultHandling import ResultHandling
print(get_all_datasets())

['cleveland', 'ionosphere', 'ecoli', 'iris', 'mammo_graphic', 'wisconsin_breast_cancer', 'australia', 'postop', 'yeast', 'spec']


# Standard method for composing result handlilngs

# Script analysis

In [54]:


if 1:
    job = 'bagboost'
    experiment_name = 'bagboost_experiment'
    RH = ResultHandling(job, experiment_name, height=10, width = 10, box_label_size=30, target_dir='results_finished')
    for train in [True, False]:
        skipped_datasets = []
        latex_data = [] # at the end hsould be (ndatasets, nmodels, nseeds)
        training_string = 'train' if train else 'test'
        
        for ds in get_all_datasets():
            # Per model train
            plot_labels, plots = [], []
            for model_nam in RH.model_names:
                plots.append(RH.get_cond_data(mgen=False, train=train, model_name=model_nam, dataset_name=ds,col='full_acc'))
                plot_labels.append(model_nam)
            RH.box_plot(boxes=plots, labels=plot_labels, save=True, draw=False, figname=f'{job}_{ds}_{training_string}')
            if len(np.array(plots).shape) == 2:
                latex_data.append(np.array(plots))
            else:
                skipped_datasets.append(ds)
                
        # Create table 
        np_latex_data = np.array(latex_data)
        np_latex_data = np.mean(np_latex_data, axis=2)
        df = pd.DataFrame(data=np_latex_data, columns=RH.model_names)
        print(skipped_datasets)
        df.index = [ds for ds in get_all_datasets() if ds not in skipped_datasets]
        with open(f'tables/{job}_{training_string}.tex','w') as file:
            file.write(df.to_latex())



['cleveland']
['cleveland']


## Saving fast bag vs baseline gp vs basline bag

In [51]:


if 1:
    job = 'fastbag_ham'
    experiment_name = 'fastbag'
    RH = ResultHandling(job, experiment_name, height=10, width = 10, box_label_size=30, target_dir='results_finished')
    other = ResultHandling('bagboost', 'bagboost_experiment', height=10, width = 10, box_label_size=30, target_dir='results_finished')
    for train in [True, False]:
        skipped_datasets = []
        latex_data = [] # at the end hsould be (ndatasets, nmodels, nseeds)
        training_string = 'train' if train else 'test'
        for ds in get_all_datasets():
            # Per model train
            plot_labels, plots = [], []
            for model_nam in other.model_names:
                if model_nam in ['GP', 'baggp']:
                    plots.append(other.get_cond_data(mgen=False, train=train, model_name=model_nam, dataset_name=ds,col='full_acc'))
                    plot_labels.append(model_nam)
            for model_nam in RH.model_names:
                plots.append(RH.get_cond_data(mgen=False, train=train, model_name=model_nam, dataset_name=ds,col='full_acc'))
                if model_nam == 'fastbaggp':
                    model_nam = 'rfbag'
                if model_nam == 'ham_fastbaggp':
                    model_nam = 'hrfbag'
                plot_labels.append(model_nam)
            RH.box_plot(boxes=plots, labels=plot_labels, save=True, draw=False, figname=f'{job}_{ds}_{training_string}')
            if len(np.array(plots).shape) == 2:
                latex_data.append(np.array(plots))
            else:
                skipped_datasets.append(ds)
                
        # Create table 
        np_latex_data = np.array(latex_data)
        np_latex_data = np.mean(np_latex_data, axis=2)
        df = pd.DataFrame(data=np_latex_data, columns=['gp', 'baggp', 'rfbag', 'rfham'])
        print(skipped_datasets)
        df.index = [ds for ds in get_all_datasets() if ds not in skipped_datasets]
        with open(f'tables/{job}_{training_string}.tex','w') as file:
            file.write(df.to_latex())
        
        
        



['mammo_graphic']
['mammo_graphic']


# PCA analysis


In [43]:
if 1:
    job = 'pca'
    base = ResultHandling('bagboost', 'bagboost_experiment', height=10, width = 10, box_label_size=30, target_dir='results_finished')
    pca = ResultHandling('pca', 'dr', height=10, width = 10, box_label_size=30, target_dir='results_finished')
    
    for train in [True, False]:
        
        latex_data = [] # at the end hsould be (ndatasets, nmodels, nseeds)
        training_string = 'train' if train else 'test'
        skipped_datasets = []
        for ds in get_all_datasets():
            # Per model train
            plot_labels, plots = [], []
            for model_nam in base.model_names:
                if model_nam in ['GP', 'baggp']:
                    plots.append(base.get_cond_data(mgen=False, train=train, model_name=model_nam, dataset_name=ds,col='full_acc'))
                    plot_labels.append(model_nam)

            for i in range(4):
                j = i + 1
                plots.append(pca.get_cond_data(mgen=False, train=train, model_name='pcabag', dataset_name=f'{ds}_{j}',col='full_acc'))
                plot_labels.append(f'pca_{j}')
            pca.box_plot(boxes=plots, labels=plot_labels, save=True, draw=False, figname=f'{job}_{ds}_{training_string}')
            
            # 
            if len(np.array(plots).shape) == 2:
                latex_data.append(np.array(plots))
            else:
                skipped_datasets.append(ds)

        # Create table 
        np_latex_data = np.array(latex_data)
        np_latex_data = np.mean(np_latex_data, axis=2)
        df = pd.DataFrame(data=np_latex_data, columns=['gp', 'baggp', 'pca1', 'pca2', 'pca3', 'pca4'])
        df.index = [ds for ds in get_all_datasets() if ds not in skipped_datasets]
        with open(f'tables/{job}_{training_string}.tex','w') as file:
            file.write(df.to_latex())


# Analysis bag m3gp

In [39]:
if 1:
    job = 'm3gpbag'
    base = ResultHandling('bagboost', 'bagboost_experiment', height=10, width = 10, box_label_size=30, target_dir='results_finished')
    m3gpbag = ResultHandling('m3gpbag', 'm3gp_bag', height=10, width = 10, box_label_size=30, target_dir='results_finished')
    
    for train in [True, False]:
        
        latex_data = [] # at the end hsould be (ndatasets, nmodels, nseeds)
        training_string = 'train' if train else 'test'
        skipped_datasets = []
        
        for ds in get_all_datasets():
            # Per model train
            plot_labels, plots = [], []
            for model_nam in base.model_names:
                if model_nam in ['GP', 'baggp']:
                    plots.append(base.get_cond_data(mgen=False, train=train, model_name=model_nam, dataset_name=ds,col='full_acc'))
                    plot_labels.append(model_nam)
            for model_nam in m3gpbag.model_names:
                plots.append(m3gpbag.get_cond_data(mgen=False, train=train, model_name=model_nam, dataset_name=ds,col='full_acc'))
                plot_labels.append(model_nam)
            m3gpbag.box_plot(boxes=plots, labels=plot_labels, save=True, draw=False, figname=f'{job}_{ds}_{training_string}')
            if len(np.array(plots).shape) == 2:
                latex_data.append(np.array(plots))
            else:
                skipped_datasets.append(ds)
                
        # Create table 
        np_latex_data = np.array(latex_data)
        np_latex_data = np.mean(np_latex_data, axis=2)
        df = pd.DataFrame(data=np_latex_data, columns=['GP', 'BagModel', 'm3gp'])
        df.index = [ds for ds in get_all_datasets() if ds not in skipped_datasets]
        with open(f'tables/{job}_{training_string}.tex','w') as file:
            file.write(df.to_latex())


# Analysis bag MOGP

In [36]:
if 1:
    job = 'bagmogp'
    base = ResultHandling('bagboost', 'bagboost_experiment', height=10, width = 10, box_label_size=30, target_dir='results_finished')
    mogpbag = ResultHandling('bagmogp', 'bagmogp', height=10, width = 10, box_label_size=30, target_dir='results_finished')
    mogp500 = ResultHandling('mogp500', 'bagmogp', height=10, width = 10, box_label_size=30, target_dir='results_finished')
    
    for train in [True, False]:
        latex_data = [] # at the end hsould be (ndatasets, nmodels, nseeds)
        training_string = 'train' if train else 'test'
        skipped_datasets = []
        
        for ds in get_all_datasets():
            plot_labels, plots = [], []

            for model_nam in base.model_names:
                if model_nam in ['GP', 'baggp']:
                    plots.append(base.get_cond_data(mgen=False, train=train, model_name=model_nam, dataset_name=ds,col='full_acc'))
                    plot_labels.append(model_nam)

            for model_nam in mogpbag.model_names:
                plots.append(mogpbag.get_cond_data(mgen=False, train=train, model_name=model_nam, dataset_name=ds,col='full_acc'))
                plot_labels.append(model_nam)

            for model_nam in mogp500.model_names:
                plots.append(mogp500.get_cond_data(mgen=False, train=train, model_name=model_nam, dataset_name=ds,col='full_acc'))
                plot_labels.append(model_nam)
            mogpbag.box_plot(boxes=plots, labels=plot_labels, save=True, draw=False, figname=f'{job}_{ds}_{training_string}')

            if len(np.array(plots).shape) == 2:
                latex_data.append(np.array(plots))
            else:
                skipped_datasets.append(ds)
        
        # Create table 
        np_latex_data = np.array(latex_data)
        np_latex_data = np.mean(np_latex_data, axis=2)
        df = pd.DataFrame(data=np_latex_data, columns=['GP', 'BagModel', 'mogp250', 'mogp500'])
        df.index = [ds for ds in get_all_datasets() if ds not in skipped_datasets]
        with open(f'tables/{job}_{training_string}.tex','w') as file:
            file.write(df.to_latex())


# Script for generating table and wilson cox test

In [7]:
f = RH.get_model_vs_dataset_dataframe(training=True)
print(f.to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &        GP &     baggp &    nichgp &      ccgp \\
\midrule
cleveland               &  0.883414 &  0.840419 &       NaN &  0.922383 \\
ionosphere              &  0.944082 &  0.905034 &  0.953061 &  0.988707 \\
ecoli                   &  0.779858 &  0.518723 &  0.737872 &  0.766241 \\
iris                    &  0.984762 &  0.983810 &  0.983492 &  0.974921 \\
mammo\_graphic           &  0.847619 &  0.819966 &  0.844865 &  0.860700 \\
wisconsin\_breast\_cancer &  0.982078 &  0.955439 &  0.974407 &  0.984868 \\
australia               &  0.897930 &  0.854727 &  0.887371 &  0.928640 \\
postop                  &  0.872131 &  0.879235 &  0.868306 &  0.915847 \\
yeast                   &  0.425209 &  0.322929 &  0.404303 &  0.443256 \\
spec                    &  0.826523 &  0.782079 &  0.826523 &  0.896953 \\
\bottomrule
\end{tabular}



In [8]:
f = RH.get_model_vs_dataset_dataframe(training=False)
print(f.to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &        GP &     baggp &    nichgp &      ccgp \\
\midrule
cleveland               &  0.791852 &  0.776667 &       NaN &  0.779630 \\
ionosphere              &  0.866038 &  0.821384 &  0.896855 &  0.897484 \\
ecoli                   &  0.734653 &  0.501320 &  0.705611 &  0.703960 \\
iris                    &  0.947407 &  0.953333 &  0.950370 &  0.928889 \\
mammo\_graphic           &  0.821553 &  0.802276 &  0.829050 &  0.825837 \\
wisconsin\_breast\_cancer &  0.961138 &  0.940325 &  0.957886 &  0.959837 \\
australia               &  0.851369 &  0.825765 &  0.851852 &  0.848953 \\
postop                  &  0.628395 &  0.590123 &  0.628395 &  0.597531 \\
yeast                   &  0.401495 &  0.311734 &  0.392601 &  0.413303 \\
spec                    &  0.683128 &  0.649794 &  0.702469 &  0.688477 \\
\bottomrule
\end{tabular}



In [9]:
RH.wilcoxon_test()

['baggp', 'GP', RanksumsResult(statistic=-3.1096223731159465, pvalue=0.0009366333448071793)]
['nichgp', 'GP', RanksumsResult(statistic=0.5317782348576343, pvalue=0.7025602005035679)]
['nichgp', 'baggp', RanksumsResult(statistic=3.3804082908359097, pvalue=0.9996381088767804)]
['ccgp', 'GP', RanksumsResult(statistic=-0.050633808710991254, pvalue=0.4798086609564054)]
['ccgp', 'baggp', RanksumsResult(statistic=2.946887666979691, pvalue=0.9983950510652889)]
['ccgp', 'nichgp', RanksumsResult(statistic=-0.6181157931031984, pvalue=0.2682495062229687)]


In [10]:
q = np.array([1,2,3])
print(np.mean(q))
print(type(float(np.mean(q))))

2.0
<class 'float'>


# Fast GP

In [11]:
RH.model_names

['GP', 'baggp', 'nichgp', 'ccgp']

In [9]:
# For a given experiment we use training and test per experiment 

RH = ResultHandling(job, experiment_name, height=10, width = 10, box_label_size=30, target_dir='results_finished')
f = RH.get_model_vs_dataset_dataframe(training=False)
print(type(f.to_latex()))
with open('sometexfile.tex','w') as file:
    file.write(f.to_latex())

<class 'str'>


In [19]:
print(pd.DataFrame(data=np.array(plots).transpose()).to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &         0 &         1 &         2 &         3 \\
\midrule
0  &  0.617284 &  0.641975 &  0.592593 &  0.728395 \\
1  &  0.728395 &  0.666667 &  0.654321 &  0.654321 \\
2  &  0.666667 &  0.703704 &  0.679012 &  0.753086 \\
3  &  0.641975 &  0.604938 &  0.666667 &  0.629630 \\
4  &  0.592593 &  0.580247 &  0.604938 &  0.666667 \\
5  &  0.641975 &  0.629630 &  0.666667 &  0.679012 \\
6  &  0.679012 &  0.703704 &  0.629630 &  0.716049 \\
7  &  0.629630 &  0.567901 &  0.728395 &  0.641975 \\
8  &  0.777778 &  0.703704 &  0.604938 &  0.740741 \\
9  &  0.641975 &  0.654321 &  0.716049 &  0.703704 \\
10 &  0.691358 &  0.654321 &  0.740741 &  0.777778 \\
11 &  0.790123 &  0.654321 &  0.604938 &  0.654321 \\
12 &  0.765432 &  0.777778 &  0.802469 &  0.765432 \\
13 &  0.604938 &  0.679012 &  0.679012 &  0.679012 \\
14 &  0.679012 &  0.629630 &  0.703704 &  0.740741 \\
15 &  0.679012 &  0.641975 &  0.703704 &  0.691358 \\
16 &  0.654321 &  0.617284 &  0.679012 & 